In [18]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [19]:
background = None
acc_weight = 0.5
roi_top = 50
roi_bottom = 300
roi_right = 200
roi_left = 500

In [20]:
def cal_acc_avg(frame, acc_weight):
    global background
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, acc_weight)

In [21]:
def segment(frame, threshold=25):
    global background
    diff = cv2.absdiff(background.astype("uint8"), frame)
    _, thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hand_segment = max(contours, key=cv2.contourArea) if contours else None
    return thresh, hand_segment

In [13]:
def count_fingers(thresh, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    top, bottom, left, right = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0]), \
                              tuple(conv_hull[conv_hull[:, :, 1].argmax()][0]), \
                              tuple(conv_hull[conv_hull[:, :, 0].argmin()][0]), \
                              tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    cX, cY = (left[0] + right[0]) // 2, (top[1] + bottom[1]) // 2
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    max_distance = distance.max()
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)
    circular_mask = np.zeros(thresh.shape[:2], dtype="uint8")
    cv2.circle(circular_mask, (cX, cY), radius, 255, 10)
    circular_roi = cv2.bitwise_and(thresh, thresh, mask=circular_mask)
    contours, _ = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    count = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        limit_points = ((circumference * 0.25) > contour.shape[0])
        if out_of_wrist and limit_points:
            count += 1
    return count

In [27]:
import cv2

cam = cv2.VideoCapture(0)

num_frame = 0
hand = None

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()

    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    if num_frame < 60:
        cal_acc_avg(gray, acc_weight)
        if num_frame <= 59:
            cv2.putText(frame_copy, "Calculating Background Avg...", (200, 400),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Finger Count : ", frame_copy)
    else:
        hand = segment(gray)

    
    if hand is not None:
        thresh, hand_segment = hand
        if hand_segment is not None:
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0), 5)
            fingers = count_fingers(thresh, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Threshold", thresh)

    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0, 0, 255), 5)
    num_frame += 1

    cv2.imshow("Finger Count", frame_copy)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()

In [26]:
cam.release()
cv2.destroyAllWindows()